In [1]:
import json

import numpy as np
import pandas as pd

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
sample_size = 1

## COCO

### Inspect json

In [3]:
with open('./instances_val2017.json', 'r') as f:
    ann_dict = json.load(f)

In [4]:
ann_dict.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [5]:
len(ann_dict['images'])

5000

In [6]:
# id_list = []
# for i in range(sample_size):
#     id_list.append(ann_dict['annotations'][i]['id'])

In [7]:
len(ann_dict['annotations'])

36781

### Create small GT set

In [8]:
small_ann_dict = ann_dict['annotations'][0:sample_size]
ann_dict['annotations'] = small_ann_dict
assert len(ann_dict['annotations']) == sample_size

In [9]:
image_ids = set([item['image_id'] for item in small_ann_dict[0:sample_size]])

In [10]:
df = pd.DataFrame(ann_dict['images'])

In [11]:
small_images = df[df['id'].isin(image_ids)].to_dict('records')

In [12]:
for i in range(len(image_ids)):
    small_images[i]['height'] = 0
    small_images[i]['width'] = 0

In [13]:
ann_dict['images'] = small_images

In [14]:
with open('./test.json', 'w') as f:
    json.dump(ann_dict, f)

### Load GT

In [15]:
# annFile = './instances_val2017.json'
annFile = './test.json'
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [16]:
all_id = list(cocoGt.anns.keys())
#assert len(all_id) == 36781
assert len(all_id) == sample_size

### Generate fake pred and save as json

In [17]:
ids = all_id[0:sample_size]

In [18]:
anns = cocoGt.loadAnns(ids)

In [19]:
imgIds = []

fake_pred = []
np.random.seed(seed=256)
scores = (0.97 - 0.4)* np.random.random((500,)) + 0.4

for i, annotation in enumerate(anns):
    fake_pred.append({
        'image_id': annotation['image_id'],
        'category_id': annotation['category_id'],
        'bbox': (annotation['bbox'] + 2 * ((1 - (-1)) * np.random.random(4) + 1)).tolist(),
        'score': scores[i]
    })
    imgIds.append(annotation['image_id'])

In [20]:
with open('fake_pred.json', 'wt') as f:
    json.dump(fake_pred, f)

### Load fake pred

In [21]:
#initialize COCO detections api
cocoDt=cocoGt.loadRes('./fake_pred.json')

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


In [22]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt, 'bbox')
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
rc: [1.]
pr: [1.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
rc: [0.]
pr: [0.]
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.09999999999999999167332731531133
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.99999999999999988897769753748435
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.00000000000000000000000000000000


In [23]:
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.09999999999999999167332731531133
#  Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.99999999999999988897769753748435
#  Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.00000000000000000000000000000000
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.09999999999999999167332731531133
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.00000000000000000000000000000000
#  Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.00000000000000000000000000000000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.10000000000000000555111512312578
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.10000000000000000555111512312578
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.10000000000000000555111512312578
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.10000000000000000555111512312578
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.00000000000000000000000000000000
#  Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.00000000000000000000000000000000

## FE mAP

### Create pred

In [24]:
batch_size = len(fake_pred)
pred_bbox_per_image = 1

In [25]:
pred_shape = (pred_bbox_per_image, 6)
pred = [np.zeros(pred_shape)] * batch_size

In [26]:
for i, item in enumerate(fake_pred):
    pred[i] = np.expand_dims(
        np.concatenate([fake_pred[i]['bbox'], [fake_pred[i]['category_id']], [fake_pred[i]['score']]]), axis=0)

In [27]:
assert len(pred) == batch_size

In [28]:
for item in pred:
    assert item.shape == pred_shape

### Create bbox

In [29]:
len(anns)

1

In [30]:
anns[0].keys()

dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id', 'ignore', '_ignore'])

In [31]:
print(anns[i]['bbox'])
print(anns[i]['category_id'])

[473.07, 395.93, 38.65, 28.67]
18


In [32]:
for i in range(batch_size):
    assert np.array(anns[i]['bbox']).shape == (4, )
    assert isinstance(anns[i]['category_id'], int)

In [33]:
bbox_per_image = 1

In [34]:
bbox_shape = (batch_size, bbox_per_image, 5)
bbox = np.zeros(bbox_shape)

In [35]:
for i, item in enumerate(anns):
    bbox[i, 0] = np.concatenate([anns[i]['bbox'], [anns[i]['category_id']]])

In [36]:
assert bbox.shape == bbox_shape

In [37]:
data = {}
data['pred'] = pred
data['bbox'] = bbox

### Calculate mAP

In [38]:
from fastestimator.trace.metric import MeanAveragePrecision

In [39]:
mAP = MeanAveragePrecision(num_classes=90)

In [40]:
mAP.on_epoch_begin(data)

In [41]:
mAP.on_batch_begin(data)

In [42]:
mAP.on_batch_end(data)

In [43]:
mAP.on_epoch_end(data)

recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]
recall: [0.]
precision: [0.]


In [44]:
data.keys()

dict_keys(['pred', 'bbox', 'mAP', 'AP50', 'AP75'])

In [45]:
mAP = data['mAP']
print(f"{mAP:.32f}")

0.00000000000000000000000000000000


In [46]:
ap50 = data['AP50']
print(f"{ap50:.32f}")

0.00000000000000000000000000000000


In [47]:
ap75 = data['AP75']
print(f"{ap75:.32f}")

0.00000000000000000000000000000000
